# Прекод

# Сборный проект-4

Вам поручено разработать демонстрационную версию поиска изображений по запросу.

Для демонстрационной версии нужно обучить модель, которая получит векторное представление изображения, векторное представление текста, а на выходе выдаст число от 0 до 1 — покажет, насколько текст и картинка подходят друг другу.

### Описание данных

Данные доступны по [ссылке](https://code.s3.yandex.net/datasets/dsplus_integrated_project_4.zip).

В файле `train_dataset.csv` находится информация, необходимая для обучения: имя файла изображения, идентификатор описания и текст описания. Для одной картинки может быть доступно до 5 описаний. Идентификатор описания имеет формат `<имя файла изображения>#<порядковый номер описания>`.

В папке `train_images` содержатся изображения для тренировки модели.

В файле `CrowdAnnotations.tsv` — данные по соответствию изображения и описания, полученные с помощью краудсорсинга. Номера колонок и соответствующий тип данных:

1. Имя файла изображения.
2. Идентификатор описания.
3. Доля людей, подтвердивших, что описание соответствует изображению.
4. Количество человек, подтвердивших, что описание соответствует изображению.
5. Количество человек, подтвердивших, что описание не соответствует изображению.

В файле `ExpertAnnotations.tsv` содержатся данные по соответствию изображения и описания, полученные в результате опроса экспертов. Номера колонок и соответствующий тип данных:

1. Имя файла изображения.
2. Идентификатор описания.

3, 4, 5 — оценки трёх экспертов.

Эксперты ставят оценки по шкале от 1 до 4, где 1 — изображение и запрос совершенно не соответствуют друг другу, 2 — запрос содержит элементы описания изображения, но в целом запрос тексту не соответствует, 3 — запрос и текст соответствуют с точностью до некоторых деталей, 4 — запрос и текст соответствуют полностью.

В файле `test_queries.csv` находится информация, необходимая для тестирования: идентификатор запроса, текст запроса и релевантное изображение. Для одной картинки может быть доступно до 5 описаний. Идентификатор описания имеет формат `<имя файла изображения>#<порядковый номер описания>`.

В папке `test_images` содержатся изображения для тестирования модели.

## 1. Исследовательский анализ данных

Наш датасет содержит экспертные и краудсорсинговые оценки соответствия текста и изображения.

В файле с экспертными мнениями для каждой пары изображение-текст имеются оценки от трёх специалистов. Для решения задачи вы должны эти оценки агрегировать — превратить в одну. Существует несколько способов агрегации оценок, самый простой — голосование большинства: за какую оценку проголосовала большая часть экспертов (в нашем случае 2 или 3), та оценка и ставится как итоговая. Поскольку число экспертов меньше числа классов, может случиться, что каждый эксперт поставит разные оценки, например: 1, 4, 2. В таком случае данную пару изображение-текст можно исключить из датасета.

Вы можете воспользоваться другим методом агрегации оценок или придумать свой.

В файле с краудсорсинговыми оценками информация расположена в таком порядке:

1. Доля исполнителей, подтвердивших, что текст **соответствует** картинке.
2. Количество исполнителей, подтвердивших, что текст **соответствует** картинке.
3. Количество исполнителей, подтвердивших, что текст **не соответствует** картинке.

После анализа экспертных и краудсорсинговых оценок выберите либо одну из них, либо объедините их в одну по какому-то критерию: например, оценка эксперта принимается с коэффициентом 0.6, а крауда — с коэффициентом 0.4.

Ваша модель должна возвращать на выходе вероятность соответствия изображения тексту, поэтому целевая переменная должна иметь значения от 0 до 1.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from scipy.stats import mode

import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import os
from tqdm import tqdm

from transformers import BertTokenizer, BertModel

from sklearn.model_selection import GroupShuffleSplit

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import matplotlib.image as mpimg
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn as nn

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def data_exploration_func(df_list):
    for df_current in df_list:
        print(f'Изучение данных датафрейма "{df_current.name}"\n')
        print(f'Первые 10 строк "{df_current.name}"')
        display(df_current.head(10))
        print('\n')
        print(f'"{df_current.name}" состоит из:')
        print(f'{df_current.shape[0]} строк')
        print(f'{df_current.shape[1]} столбцов')
        print('\n')
        print(f'Общая информация о "{df_current.name}"\n')
        df_current.info()
        print('\n')
        print(f'Описательная статистика "{df_current.name}"')
        display(df_current.describe(include='all').T)
        print('\n')

In [4]:
RANDOM_STATE=42

In [5]:
from google.colab import drive
drive.mount('/content/drive')
df_train = pd.read_csv('/content/drive/MyDrive/data/train_dataset.csv', sep=',')
df_crowdAnnotations = pd.read_csv('/content/drive/MyDrive/data/CrowdAnnotations.tsv',
                                  sep='\t',
                                  names=['image', 'query_id', 'desc_perc', 'n_match', 'n_no_match'],
                                  index_col=False)
df_expertAnnotations = pd.read_csv('/content/drive/MyDrive/data/ExpertAnnotations.tsv',
                                   sep='\t',
                                   names=['image', 'query_id', 'ex_1', 'ex_2', 'ex_3'],
                                   index_col=False)
df_test = pd.read_csv('/content/drive/MyDrive/data/test_queries.csv', sep='|')
image_folder = '/content/drive/MyDrive/data/train_images'
test_queries = pd.read_csv('/content/drive/MyDrive/data/test_queries.csv', sep='|')
test_images = '/content/drive/MyDrive/data/test_images'

Mounted at /content/drive


In [6]:
df_train.name= 'df_train'
df_crowdAnnotations.name = 'df_crowdAnnotations'
df_expertAnnotations.name = 'df_expertAnnotations'
df_test.name = 'df_test'

In [7]:
df_list = [df_train, df_crowdAnnotations, df_expertAnnotations, df_test]
data_exploration_func(df_list)

Изучение данных датафрейма "df_train"

Первые 10 строк "df_train"


,image,query_id,query_text
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
1,1262583859_653f1469a9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
2,2447284966_d6bbdb4b6e.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
3,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
4,2621415349_ef1a7e73be.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
5,3030566410_393c36a6c5.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
6,3155451946_c0862c70cb.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
7,3222041930_f642f49d28.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
8,343218198_1ca90e0734.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
9,3718964174_cb2dc1615e.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...




"df_train" состоит из:
5822 строк
3 столбцов


Общая информация о "df_train"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image       5822 non-null   object
 1   query_id    5822 non-null   object
 2   query_text  5822 non-null   object
dtypes: object(3)
memory usage: 136.6+ KB


Описательная статистика "df_train"


,count,unique,top,freq
image,5822,1000,754852108_72f80d421f.jpg,10
query_id,5822,977,2600867924_cd502fc911.jpg#2,34
query_text,5822,977,"Two dogs , one brown and white and one black a...",34




Изучение данных датафрейма "df_crowdAnnotations"

Первые 10 строк "df_crowdAnnotations"


...

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
image,5822,1000,3107513635_fe8a21f148.jpg,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
query_id,5822,977,2600867924_cd502fc911.jpg#2,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ex_1,5822.0,NaN,NaN,NaN,1.43662,0.787084,1.0,1.0,1.0,2.0,4.0
ex_2,5822.0,NaN,NaN,NaN,1.624356,0.856222,1.0,1.0,1.0,2.0,4.0
ex_3,5822.0,NaN,NaN,NaN,1.881999,0.904087,1.0,1.0,2.0,2.0,4.0




Изучение данных датафрейма "df_test"

Первые 10 строк "df_test"


,Unnamed: 0,query_id,query_text,image
0,0,1177994172_10d143cb8d.jpg#0,"Two blonde boys , one in a camouflage shirt an...",1177994172_10d143cb8d.jpg
1,1,1177994172_10d143cb8d.jpg#1,Two boys are squirting water guns at each other .,1177994172_10d143cb8d.jpg
2,2,1177994172_10d143cb8d.jpg#2,Two boys spraying each other with water,1177994172_10d143cb8d.jpg
3,3,1177994172_10d143cb8d.jpg#3,Two children wearing jeans squirt water at eac...,1177994172_10d143cb8d.jpg
4,4,1177994172_10d143cb8d.jpg#4,Two young boys are squirting water at each oth...,1177994172_10d143cb8d.jpg
5,5,1232148178_4f45cc3284.jpg#0,A baby girl playing at a park .,1232148178_4f45cc3284.jpg
6,6,1232148178_4f45cc3284.jpg#1,A closeup of a child on a playground with adul...,1232148178_4f45cc3284.jpg
7,7,1232148178_4f45cc3284.jpg#2,A young boy poses for a picture in front of a ...,1232148178_4f45cc3284.jpg
8,8,1232148178_4f45cc3284.jpg#3,A young girl is smiling in front of the camera...,1232148178_4f45cc3284.jpg
9,9,1232148178_4f45cc3284.jpg#4,There is a little blond hair girl with a green...,1232148178_4f45cc3284.jpg




"df_test" состоит из:
500 строк
4 столбцов


Общая информация о "df_test"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  500 non-null    int64 
 1   query_id    500 non-null    object
 2   query_text  500 non-null    object
 3   image       500 non-null    object
dtypes: int64(1), object(3)
memory usage: 15.8+ KB


Описательная статистика "df_test"


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Unnamed: 0,500.0,NaN,NaN,NaN,249.5,144.481833,0.0,124.75,249.5,374.25,499.0
query_id,500,500,989851184_9ef368e520.jpg#4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
query_text,500,500,The black dog has a toy in its mouth and a per...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
image,500,100,1177994172_10d143cb8d.jpg,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_test.sample(15)

,Unnamed: 0,query_id,query_text,image
415,415,3722572342_6904d11d52.jpg#0,a red covered boat racing across the water,3722572342_6904d11d52.jpg
397,397,3555573680_41c1540a86.jpg#2,A man on a waterski is performing a jump in th...,3555573680_41c1540a86.jpg
290,290,3330333217_1a69497a74.jpg#0,A black man in a blue shirt stands next to a b...,3330333217_1a69497a74.jpg
288,288,3287969199_08e775d896.jpg#3,A large bird spreading his wings in flight ove...,3287969199_08e775d896.jpg
300,300,3356748019_2251399314.jpg#0,Cyclists are leaping into the air whilst being...,3356748019_2251399314.jpg
253,253,3163198309_bbfe504f0a.jpg#3,People are watching a skier perform a trick in...,3163198309_bbfe504f0a.jpg
428,428,381514859_b40418d9c3.jpg#3,Two West Highland Terriers chase a red ball .,381514859_b40418d9c3.jpg
496,496,989851184_9ef368e520.jpg#1,A black dog has a dumbbell in his mouth lookin...,989851184_9ef368e520.jpg
84,84,2308256827_3c0a7d514d.jpg#4,The little boy is smiling as he crosses a rope...,2308256827_3c0a7d514d.jpg
182,182,2887171449_f54a2b9f39.jpg#2,A woman works by a sewing machine .,2887171449_f54a2b9f39.jpg


**Вывод по загрузке данных**:
- с таблицей `df_train` всё в порядке
- в таблицах `df_crowdAnnotations` и `df_expertAnnotations` первые записили превратились в названия столбцов (убрал при загрузке таблиц)
- в таблице `df_test` первый столбец повторяет `id`, поэтому его можно удалить

In [9]:
df_test = df_test.drop('Unnamed: 0', axis=1)

In [10]:
df_test.head(5)

,query_id,query_text,image
0,1177994172_10d143cb8d.jpg#0,"Two blonde boys , one in a camouflage shirt an...",1177994172_10d143cb8d.jpg
1,1177994172_10d143cb8d.jpg#1,Two boys are squirting water guns at each other .,1177994172_10d143cb8d.jpg
2,1177994172_10d143cb8d.jpg#2,Two boys spraying each other with water,1177994172_10d143cb8d.jpg
3,1177994172_10d143cb8d.jpg#3,Two children wearing jeans squirt water at eac...,1177994172_10d143cb8d.jpg
4,1177994172_10d143cb8d.jpg#4,Two young boys are squirting water at each oth...,1177994172_10d143cb8d.jpg


In [11]:
for df in df_list:
    print(f'Пропущенные значения "{df.name}":\n{df.isna().sum()}')
    print('\n')

Пропущенные значения "df_train":
image         0
query_id      0
query_text    0
dtype: int64


Пропущенные значения "df_crowdAnnotations":
image         0
query_id      0
desc_perc     0
n_match       0
n_no_match    0
dtype: int64


Пропущенные значения "df_expertAnnotations":
image       0
query_id    0
ex_1        0
ex_2        0
ex_3        0
dtype: int64


Пропущенные значения "df_test":
Unnamed: 0    0
query_id      0
query_text    0
image         0
dtype: int64




In [12]:
for df in df_list:
    print(f'Явные дубликаты "{df.name}":\n{df.duplicated().sum()}')
    print('\n')

Явные дубликаты "df_train":
0


Явные дубликаты "df_crowdAnnotations":
0


Явные дубликаты "df_expertAnnotations":
0


Явные дубликаты "df_test":
0




In [13]:
datagen = ImageDataGenerator(rescale=1./255)
sample_generator = datagen.flow_from_dataframe(
    dataframe=df_train.head(5),
    directory='/content/drive/MyDrive/data/train_images',
    x_col='image',
    y_col='query_text',
    target_size=(224, 224),
    batch_size=15,
    class_mode='raw'
)

Found 5 validated image filenames.


In [14]:
images, queries = next(sample_generator)

In [15]:
num_images = len(images)
plt.figure(figsize=(10, 5 * num_images))

for i in range(num_images):
    plt.subplot(num_images, 1, i+1)
    plt.imshow(images[i])
    plt.title(f"Query: {queries[i]}", fontsize=12)
    plt.axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [16]:
# Функция для агрегации экспертных оценок
def aggregate_scores(row):
    scores = [row['ex_1'], row['ex_2'], row['ex_3']]

    if len(set(scores)) == 3:
        return None

    modal_score = mode(scores)
    return modal_score[0]

In [17]:
# Применяем агрегацию
df_expertAnnotations['expert_score'] = df_expertAnnotations.apply(aggregate_scores, axis=1)
# Удаляем строки, где эксперты сильно разошлись (вернулся None)
df_expertAnnotations = df_expertAnnotations.dropna(subset=['expert_score'])
# Нормализуем оценку в диапазон [0, 1] (делением на 4)
df_expertAnnotations['expert_score_norm'] = df_expertAnnotations['expert_score'] / 4.0

In [18]:
df_expertAnnotations.head()

,image,query_id,ex_1,ex_2,ex_3,expert_score,expert_score_norm
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,1,1,1,1.0,0.25
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,1,1,2,1.0,0.25
2,1056338697_4f7d7ce270.jpg,3181701312_70a379ab6e.jpg#2,1,1,2,1.0,0.25
3,1056338697_4f7d7ce270.jpg,3207358897_bfa61fa3c6.jpg#2,1,2,2,2.0,0.50
4,1056338697_4f7d7ce270.jpg,3286822339_5535af6b93.jpg#2,1,1,2,1.0,0.25


In [19]:
# Вычисляем общее число голосов
df_crowdAnnotations['total_votes'] = df_crowdAnnotations['n_match'] + df_crowdAnnotations['n_no_match']
# Вычисляем "уверенность" оценки (чем больше голосов, тем увереннее)
df_crowdAnnotations['confidence'] = 1 - 1/(df_crowdAnnotations['total_votes'] + 1)
# Взвешенная оценка
df_crowdAnnotations['crowd_score_norm'] = df_crowdAnnotations['desc_perc'] * df_crowdAnnotations['confidence']

In [20]:
df_crowdAnnotations.head()

,image,query_id,desc_perc,n_match,n_no_match,total_votes,confidence,crowd_score_norm
0,1056338697_4f7d7ce270.jpg,1056338697_4f7d7ce270.jpg#2,1.0,3,0,3,0.75,0.75
1,1056338697_4f7d7ce270.jpg,114051287_dd85625a04.jpg#2,0.0,0,3,3,0.75,0.00
2,1056338697_4f7d7ce270.jpg,1427391496_ea512cbe7f.jpg#2,0.0,0,3,3,0.75,0.00
3,1056338697_4f7d7ce270.jpg,2073964624_52da3a0fc4.jpg#2,0.0,0,3,3,0.75,0.00
4,1056338697_4f7d7ce270.jpg,2083434441_a93bc6306b.jpg#2,0.0,0,3,3,0.75,0.00


In [21]:
def merge_annotations(df_expertAnnotations, df_crowdAnnotations):
    merged = pd.merge(
        df_expertAnnotations[['image', 'query_id', 'expert_score_norm']],
        df_crowdAnnotations[['image', 'query_id', 'crowd_score_norm', 'total_votes']],
        on=['image', 'query_id'],
        how='outer'  # Используем outer join для сохранения всех возможных пар
    )

    # Заполняем пропуски (если каких-то оценок нет)
    merged['expert_score_norm'] = merged['expert_score_norm'].fillna(0.5)
    merged['crowd_score_norm'] = merged['crowd_score_norm'].fillna(0.5)
    merged['total_votes'] = merged['total_votes'].fillna(1)  # Минимум 1 голос

    return merged

In [22]:
annotations_merged = merge_annotations(df_expertAnnotations, df_crowdAnnotations)

In [23]:
# 3. Функция для расчета target
def calculate_target(row):
    expert = row['expert_score_norm']
    crowd = row['crowd_score_norm']
    votes = row['total_votes']

    # Базовые веса
    expert_weight = 0.7
    crowd_weight = 0.3

    # Динамическая корректировка весов
    if votes > 10:
        crowd_weight = min(0.45, 0.3 + 0.02 * votes)
        expert_weight = 1 - crowd_weight
    elif votes < 3:
        expert_weight = 0.85
        crowd_weight = 0.15

    # Итоговый target
    target = expert_weight * expert + crowd_weight * crowd

    # Гарантируем границы [0, 1]
    return np.clip(target, 0, 1)

In [24]:
annotations_merged['target'] = annotations_merged.apply(calculate_target, axis=1)

In [25]:
df_train[['image', 'desc_num']] = df_train['query_id'].str.split('#', expand=True)
df_train['desc_num'] = df_train['desc_num'].astype(int)

In [26]:
annotations_merged['desc_num'] = annotations_merged['query_id'].str.split('#').str[1].astype(int)

In [27]:
df_train = pd.merge(
    df_train,
    annotations_merged[['image', 'desc_num', 'target']],
    on=['image', 'desc_num'],
    how='left'
)

In [28]:
df_train = df_train.drop(columns=['desc_num'])

In [29]:
df_train.head()

,image,query_id,query_text,target
0,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...,0.35
1,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...,0.35
2,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...,0.35
3,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...,0.35
4,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...,0.35


In [30]:
df_train['target'].sort_values().unique()

array([0.175     , 0.235     , 0.25      , 0.2875    , 0.35      ,
       0.39285714, 0.4       , 0.41      , 0.425     , 0.43571429,
       0.45      , 0.47      , 0.5       , 0.52142857, 0.525     ,
       0.53      , 0.55      , 0.575     , 0.585     , 0.59      ,
       0.6       , 0.6       , 0.61071429, 0.625     , 0.645     ,
       0.65357143, 0.675     , 0.69642857, 0.7       , 0.705     ,
       0.7125    , 0.725     , 0.75      , 0.775     , 0.85      ,
       0.925     , 0.94      , 0.95      ])

**Вывод**:
- исправили ошибки в данных
- агрегировали оценки экспертов и краудсорсинговые оценки
- создали `target`

## 2. Проверка данных

В некоторых странах, где работает ваша компания, действуют ограничения по обработке изображений: поисковым сервисам и сервисам, предоставляющим возможность поиска, запрещено без разрешения родителей или законных представителей предоставлять любую информацию, в том числе, но не исключительно тексты, изображения, видео и аудио, содержащие описание, изображение или запись голоса детей. Ребёнком считается любой человек, не достигший 16 лет.

В вашем сервисе строго следуют законам стран, в которых работают. Поэтому при попытке посмотреть изображения, запрещённые законодательством, вместо картинок показывается дисклеймер:

> This image is unavailable in your country in compliance with local laws
>

Однако у вас в PoC нет возможности воспользоваться данным функционалом. Поэтому все изображения, которые нарушают данный закон, нужно удалить из обучающей выборки.

In [31]:
banned_keywords = r'child|kid|baby|babies|boy|girl'

In [32]:
banned_query_ids = df_train[df_train['query_text'].str.contains(banned_keywords, case=False, na=False)]['query_id'].unique()

In [33]:
df_train_filtered = df_train[~df_train['query_id'].isin(banned_query_ids)].copy()

In [34]:
df_train_filtered.head()

,image,query_id,query_text,target
1334,3181701312_70a379ab6e.jpg,3181701312_70a379ab6e.jpg#2,A man sleeps under a blanket on a city street .,0.425
1335,3181701312_70a379ab6e.jpg,3181701312_70a379ab6e.jpg#2,A man sleeps under a blanket on a city street .,0.350
1336,3181701312_70a379ab6e.jpg,3181701312_70a379ab6e.jpg#2,A man sleeps under a blanket on a city street .,0.350
1337,3181701312_70a379ab6e.jpg,3181701312_70a379ab6e.jpg#2,A man sleeps under a blanket on a city street .,0.350
1338,3181701312_70a379ab6e.jpg,3181701312_70a379ab6e.jpg#2,A man sleeps under a blanket on a city street .,0.350


## 3. Векторизация изображений

Перейдём к векторизации изображений.

Самый примитивный способ — прочесть изображение и превратить полученную матрицу в вектор. Такой способ нам не подходит: длина векторов может быть сильно разной, так как размеры изображений разные. Поэтому стоит обратиться к свёрточным сетям: они позволяют "выделить" главные компоненты изображений. Как это сделать? Нужно выбрать какую-либо архитектуру, например ResNet-18, посмотреть на слои и исключить полносвязные слои, которые отвечают за конечное предсказание. При этом можно загрузить модель данной архитектуры, предварительно натренированную на датасете ImageNet.

In [35]:
# Загружаем модель и убираем последний классификационный слой
resnet = models.resnet18(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # удаляем последний слой
resnet.eval()  # переводим в режим инференса

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 163MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [36]:
# Трансформации такие же, как при обучении на ImageNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize до нужного размера
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [37]:
def get_image_embedding(img_path, model, transform):
    img = Image.open(img_path).convert('RGB')
    img_t = transform(img).unsqueeze(0)  # добавим размер батча
    with torch.no_grad():
        embedding = model(img_t).squeeze().numpy()
    return embedding

In [38]:
image_embeddings = {}

for image_name in tqdm(os.listdir(image_folder)):
    img_path = os.path.join(image_folder, image_name)
    try:
        emb = get_image_embedding(img_path, resnet, transform)
        image_embeddings[image_name] = emb
    except Exception as e:
        print(f'Ошибка с изображением {image_name}: {e}')

100%|██████████| 1000/1000 [02:20<00:00,  7.10it/s]


## 4. Векторизация текстов

Следующий этап — векторизация текстов. Вы можете поэкспериментировать с несколькими способами векторизации текстов:

- tf-idf
- word2vec
- \*трансформеры (например Bert)

\* — если вы изучали трансформеры в спринте Машинное обучение для текстов.


In [39]:
# Загружаем BERT-модель и токенизатор
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()  # переводим в режим инференса

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [40]:
def get_text_embedding(text, tokenizer, model):
    tokens = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=64)
    with torch.no_grad():
        output = model(**tokens)
    # Получаем эмбеддинги токенов и делаем mean pooling
    embeddings = output.last_hidden_state.squeeze(0)
    attention_mask = tokens['attention_mask'].squeeze(0)
    mask_expanded = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
    mean_pooled = torch.sum(embeddings * mask_expanded, dim=0) / torch.clamp(mask_expanded.sum(dim=0), min=1e-9)
    return mean_pooled.numpy()

In [41]:
text_embeddings = {}

for idx, row in tqdm(df_train_filtered.iterrows(), total=len(df_train_filtered)):
    text = row['query_text']
    query_id = row['query_id']

    if query_id not in text_embeddings:
        try:
            emb = get_text_embedding(text, tokenizer, bert_model)
            text_embeddings[query_id] = emb
        except Exception as e:
            print(f'Ошибка с query_id={query_id}: {e}')


100%|██████████| 223538/223538 [01:31<00:00, 2447.88it/s]


## 5. Объединение векторов

Подготовьте данные для обучения: объедините векторы изображений и векторы текстов с целевой переменной.

In [42]:
df_train_filtered_unique = df_train_filtered.drop_duplicates(subset=['image', 'query_id'])
print(len(df_train_filtered), "->", len(df_train_filtered_unique))

223538 -> 689


In [43]:
X = []
y = []

for idx, row in tqdm(df_train_filtered_unique.iterrows(), total=len(df_train_filtered_unique)):
    image_name = row['image']
    query_id = row['query_id']
    target = row['target']

    if image_name in image_embeddings and query_id in text_embeddings:
        image_emb = image_embeddings[image_name]
        text_emb = text_embeddings[query_id]
        combined = np.concatenate([image_emb, text_emb])
        X.append(combined)
        y.append(target)


100%|██████████| 689/689 [00:00<00:00, 9365.99it/s]


In [44]:
X = np.array(X)
y = np.array(y)

In [45]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (689, 1280)
y shape: (689,)


## 6. Обучение модели предсказания соответствия

Для обучения разделите датасет на тренировочную и тестовую выборки. Простое случайное разбиение не подходит: нужно исключить попадание изображения и в обучающую, и в тестовую выборки.
Для того чтобы учесть изображения при разбиении, можно воспользоваться классом [GroupShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupShuffleSplit.html) из библиотеки sklearn.model_selection.

Код ниже разбивает датасет на тренировочную и тестовую выборки в пропорции 7:3 так, что строки с одинаковым значением 'group_column' будут содержаться либо в тестовом, либо в тренировочном датасете.

```
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=42)
train_indices, test_indices = next(gss.split(X=df.drop(columns=['target']), y=df['target'], groups=df['group_column']))
train_df, test_df = df.loc[train_indices], df.loc[test_indices]

```

Какую модель использовать — выберите самостоятельно. Также вам предстоит выбрать метрику качества либо реализовать свою.

In [46]:
# Мы будем группировать по названию изображения
groups = df_train_filtered_unique['image'].values

# Разделим на 70% для тренировки и 30% для теста
gss = GroupShuffleSplit(n_splits=1, train_size=0.7, random_state=42)

# Разбиваем индексы на train и test
train_indices, test_indices = next(gss.split(X, y, groups=groups))

# Получаем тренировочный и тестовый датасеты
X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]

print(f"Размер обучающего набора: {X_train.shape}")
print(f"Размер тестового набора: {X_test.shape}")

Размер обучающего набора: (482, 1280)
Размер тестового набора: (207, 1280)


In [47]:
# Создаём пайплайн для линейной регрессии с масштабированием
pipeline_lr = make_pipeline(StandardScaler(), LinearRegression())

# Параметры для поиска
param_grid_lr = {
    'linearregression__fit_intercept': [True, False]
}

# Настроим GridSearchCV
grid_lr = GridSearchCV(pipeline_lr, param_grid_lr, cv=5, n_jobs=-1)
grid_lr.fit(X_train, y_train)

# Лучшая модель
best_lr_model = grid_lr.best_estimator_

# Прогнозируем с лучшей моделью
y_pred_best_lr = best_lr_model.predict(X_test)

# Метрики качества
mae_best_lr = mean_absolute_error(y_test, y_pred_best_lr)
r2_best_lr = r2_score(y_test, y_pred_best_lr)

print(f'Лучшая линейная регрессия - MAE: {mae_best_lr}, R2: {r2_best_lr}')

Лучшая линейная регрессия - MAE: 0.07729470649203242, R2: -1.449254326976817


In [48]:
# Параметры для поиска
param_grid_mlp = {
    'mlpregressor__hidden_layer_sizes': [(128, 64), (256, 128), (512, 256)],
    'mlpregressor__activation': ['relu', 'tanh'],
    'mlpregressor__solver': ['adam', 'sgd'],
    'mlpregressor__learning_rate': ['constant', 'adaptive'],
    'mlpregressor__max_iter': [500, 1000],
}

# Настроим GridSearchCV
grid_mlp = GridSearchCV(make_pipeline(StandardScaler(), MLPRegressor()), param_grid_mlp, cv=5, n_jobs=-1)
grid_mlp.fit(X_train, y_train)

# Лучшая модель
best_mlp_model = grid_mlp.best_estimator_

# Прогнозируем с лучшей моделью
y_pred_best_mlp = best_mlp_model.predict(X_test)

# Метрики качества
mae_best_mlp = mean_absolute_error(y_test, y_pred_best_mlp)
r2_best_mlp = r2_score(y_test, y_pred_best_mlp)

print(f'Лучшая нейронная сеть - MAE: {mae_best_mlp}, R2: {r2_best_mlp}')

Лучшая нейронная сеть - MAE: 0.2162970519008268, R2: -16.051059215206458


## 7. Тестирование модели

Настало время протестировать модель. Для этого получите эмбеддинги для всех тестовых изображений из папки `test_images`, выберите случайные 10 запросов из файла `test_queries.csv` и для каждого запроса выведите наиболее релевантное изображение. Сравните визуально качество поиска.

In [50]:
banned_query_ids = test_queries[test_queries['query_text'].str.contains(banned_keywords, case=False, na=False)]['query_id'].unique()

In [51]:
test_queries_filtered = test_queries[~test_queries['query_id'].isin(banned_query_ids)].copy()

In [53]:
test_image_embeddings = {}

for image_name in tqdm(os.listdir(test_images), desc="Processing test images"):
    img_path = os.path.join(test_images, image_name)
    try:
        emb = get_image_embedding(img_path, resnet, transform)
        test_image_embeddings[image_name] = emb
    except Exception as e:
        print(f'Ошибка с изображением {image_name}: {e}')

Processing test images: 100%|██████████| 101/101 [00:13<00:00,  7.24it/s]

Ошибка с изображением .DS_Store: cannot identify image file '/content/drive/MyDrive/data/test_images/.DS_Store'


In [54]:
test_text_embeddings = {}

# Выбираем 10 случайных запросов
random_queries = test_queries.sample(10, random_state=42)

for idx, row in tqdm(random_queries.iterrows(), total=len(random_queries), desc="Processing test queries"):
    query_id = row['query_id']
    text = row['query_text']

    try:
        emb = get_text_embedding(text, tokenizer, bert_model)
        test_text_embeddings[query_id] = emb
    except Exception as e:
        print(f'Ошибка с query_id={query_id}: {e}')

Processing test queries: 100%|██████████| 10/10 [00:00<00:00, 10.05it/s]


In [63]:
results = {}

for query_id, text_emb in tqdm(test_text_embeddings.items(), desc="Finding relevant images"):
    similarities = []

    for img_name, img_emb in test_image_embeddings.items():
        # Конкатенируем эмбеддинги как при обучении
        combined = np.concatenate([img_emb, text_emb])

        # Предсказываем релевантность с помощью лучшей модели (выберите best_lr_model или best_mlp_model)
        relevance = best_lr_model.predict(combined.reshape(1, -1))[0]
        similarities.append((img_name, relevance))

    # Сортируем по убыванию релевантности
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Сохраняем топ-5 изображений для каждого запроса
    results[query_id] = {
        'query_text': random_queries[random_queries['query_id'] == query_id]['query_text'].iloc[0],
        'top_images': similarities[:5]
    }

Finding relevant images: 100%|██████████| 10/10 [00:00<00:00, 17.31it/s]


In [64]:
def display_results(query_id, result):
    query_text = result['query_text']
    top_images = result['top_images']

    print(f"\nЗапрос: {query_text}")
    plt.figure(figsize=(15, 3))
    plt.suptitle(f"Запрос: {query_text}", y=1.1)

    for i, (img_name, score) in enumerate(top_images):
        img_path = os.path.join(test_images, img_name)
        img = Image.open(img_path)

        plt.subplot(1, 5, i+1)
        plt.imshow(img)
        plt.title(f"Score: {score:.2f}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [65]:
for query_id, result in results.items():
    display_results(query_id, result)

Output hidden; open in https://colab.research.google.com to view.

## 8. Выводы

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Исследовательский анализ данных выполнен
- [ ]  Проверены экспертные оценки и краудсорсинговые оценки
- [ ]  Из датасета исключены те объекты, которые выходят за рамки юридических ограничений
- [ ]  Изображения векторизованы
- [ ]  Текстовые запросы векторизованы
- [ ]  Данные корректно разбиты на тренировочную и тестовую выборки
- [ ]  Предложена метрика качества работы модели
- [ ]  Предложена модель схожести изображений и текстового запроса
- [ ]  Модель обучена
- [ ]  По итогам обучения модели сделаны выводы
- [ ]  Проведено тестирование работы модели
- [ ]  По итогам тестирования визуально сравнили качество поиска